

---

# **Chapter 10: State Management**

## **Opening Context**
Objects often change behavior fundamentally as they transition through lifecycle phases—TCP connections move from Closed to Established, UI widgets transition from Enabled to Disabled, and document editors cycle through Editing, Saved, and Dirty states. Implementing these transitions with conditional logic leads to unmaintainable monolithic classes. Simultaneously, users demand undo functionality, compilers must traverse abstract syntax trees without modifying node classes, and domain-specific languages require interpretation engines. This chapter addresses four sophisticated patterns: State for behavior-switching objects, Memento for state capture and restoration, Visitor for extending operations across object structures, and Interpreter for evaluating language grammar.

---

## **10.1 State Pattern: Objects Changing Behavior with State**

### **Intent**
*Allow an object to alter its behavior when its internal state changes. The object will appear to change its class.*

### **The Problem**
Consider a `TCPConnection` class that behaves differently based on its current state (Closed, Listening, Established). The naive approach uses conditional statements:

```typescript
// ❌ STATEFUL CONDITIONALS: Unmaintainable and violates SRP
class TCPConnection {
    private state: string = "CLOSED";
    
    open(): void {
        if (this.state === "CLOSED") {
            console.log("Opening connection...");
            this.state = "ESTABLISHED";
        } else if (this.state === "ESTABLISHED") {
            console.log("Already open");
        } else if (this.state === "LISTENING") {
            throw new Error("Cannot open while listening");
        }
    }
    
    close(): void {
        if (this.state === "ESTABLISHED") {
            console.log("Closing connection...");
            this.state = "CLOSED";
        } else if (this.state === "CLOSED") {
            console.log("Already closed");
        }
    }
    
    send(data: string): void {
        if (this.state !== "ESTABLISHED") {
            throw new Error("Cannot send: connection not established");
        }
        console.log(`Sending: ${data}`);
    }
    // Each new state requires modifying every method!
}
```

**Problems**:
- **Single Responsibility Violation**: Class manages both connection logic AND state transition rules.
- **Complexity Explosion**: Adding states requires modifying all existing methods.
- **Testing Difficulty**: Must test all state combinations within one class.

### **The Solution: State Encapsulation**

The State pattern encapsulates state-specific behavior into separate classes, delegating state-dependent operations to the current state object.

```typescript
/**
 * STATE INTERFACE
 * Declares methods that represent state-specific behavior.
 * These correspond to requests that depend on the object's state.
 */
interface TCPState {
    open(connection: TCPConnection): void;
    close(connection: TCPConnection): void;
    send(connection: TCPConnection, data: string): void;
    getName(): string;
}

/**
 * CONTEXT
 * Maintains an instance of a Concrete State subclass.
 * Delegates state-specific requests to the current state object.
 */
class TCPConnection {
    private state: TCPState;
    
    constructor() {
        this.state = new ClosedState(); // Initial state
    }
    
    // State transition method (can be called by states or context)
    setState(state: TCPState): void {
        console.log(`Transitioning: ${this.state.getName()} -> ${state.getName()}`);
        this.state = state;
    }
    
    getStateName(): string {
        return this.state.getName();
    }
    
    // Delegates to state object
    open(): void {
        this.state.open(this);
    }
    
    close(): void {
        this.state.close(this);
    }
    
    send(data: string): void {
        this.state.send(this, data);
    }
}

/**
 * CONCRETE STATES
 * Each subclass implements behavior specific to a state of the Context.
 */
class ClosedState implements TCPState {
    open(connection: TCPConnection): void {
        console.log("[Closed] Opening connection...");
        connection.setState(new EstablishedState());
    }
    
    close(connection: TCPConnection): void {
        console.log("[Closed] Already closed");
    }
    
    send(connection: TCPConnection, data: string): void {
        throw new Error("[Closed] Cannot send: connection is closed");
    }
    
    getName(): string {
        return "CLOSED";
    }
}

class EstablishedState implements TCPState {
    open(connection: TCPConnection): void {
        console.log("[Established] Already open");
    }
    
    close(connection: TCPConnection): void {
        console.log("[Established] Closing connection gracefully...");
        connection.setState(new ClosedState());
    }
    
    send(connection: TCPConnection, data: string): void {
        console.log(`[Established] Sending data: ${data}`);
    }
    
    getName(): string {
        return "ESTABLISHED";
    }
}

class ListeningState implements TCPState {
    open(connection: TCPConnection): void {
        throw new Error("[Listening] Cannot open: already listening for connections");
    }
    
    close(connection: TCPConnection): void {
        console.log("[Listening] Stopping listener...");
        connection.setState(new ClosedState());
    }
    
    send(connection: TCPConnection, data: string): void {
        // Transition to Established first (simplified)
        console.log("[Listening] Accepting connection first...");
        connection.setState(new EstablishedState());
        connection.send(data); // Delegate to new state
    }
    
    getName(): string {
        return "LISTENING";
    }
}

// Usage
const conn = new TCPConnection();
console.log(`Initial state: ${conn.getStateName()}`);

// conn.send("Hello"); // Throws error: CLOSED cannot send

conn.open(); // Transitions to ESTABLISHED
conn.send("Hello World"); // Works
conn.open(); // Already open

conn.close(); // Transitions to CLOSED
```

**Code Explanation**:

- **Context** (`TCPConnection`): Maintains a reference to the current State object. Defines the interface of interest to clients. Delegates state-specific behavior to the current State.
- **State** (`TCPState`): Interface defining state-specific methods. Every concrete state implements behavior appropriate for that state.
- **Concrete States** (`ClosedState`, `EstablishedState`, `ListeningState`): Implement state-specific behavior. Control transitions (e.g., only `EstablishedState` can transition to `ClosedState` in some implementations).

**State Transitions**:
Transitions can be handled by:
1.  **Context**: Centralizes transition logic but couples Context to all States.
2.  **States**: Each state decides which state follows it (shown in example). More distributed but natural—`EstablishedState` knows it can transition to `ClosedState`.

### **State vs. Strategy**
Both patterns use composition to vary behavior, but:
- **Strategy**: Client chooses algorithm based on configuration/optimization. Usually no state transitions between strategies.
- **State**: Object transitions between states based on internal lifecycle or external events. States know about other states (for transitions).

### **Real-World Application: UI State Machines**

```typescript
// Document editing states
interface DocumentState {
    publish(doc: Document): void;
    edit(doc: Document): void;
    review(doc: Document): void;
}

class DraftState implements DocumentState {
    publish(doc: Document): void {
        console.log("Publishing draft...");
        doc.setState(new PublishedState());
    }
    
    edit(doc: Document): void {
        console.log("Editing draft content");
    }
    
    review(doc: Document): void {
        console.log("Sending for review");
        doc.setState(new ReviewState());
    }
}

class ReviewState implements DocumentState {
    publish(doc: Document): void {
        console.log("Approved and published");
        doc.setState(new PublishedState());
    }
    
    edit(doc: Document): void {
        console.log("Rejected: returning to draft");
        doc.setState(new DraftState());
    }
    
    review(doc: Document): void {
        console.log("Already under review");
    }
}

class PublishedState implements DocumentState {
    publish(doc: Document): void {
        console.log("Already published");
    }
    
    edit(doc: Document): void {
        console.log("Creating new draft version");
        doc.setState(new DraftState());
    }
    
    review(doc: Document): void {
        console.log("Cannot review published document");
    }
}
```

---

## **10.2 Memento Pattern: Implementing Undo/History**

### **Intent**
*Without violating encapsulation, capture and externalize an object's internal state so that the object can be restored to this state later.*

### **The Problem**
Implementing undo functionality requires storing previous states. However, exposing internal state breaks encapsulation:

```typescript
// ❌ ENCAPSULATION VIOLATION: Exposing internals for undo
class Editor {
    content: string = ""; // Must be public for undo system to save it!
    cursorPosition: number = 0;
    selection: string = "";
    
    // Anyone can modify state directly, breaking invariants
}

// Undo system forced to depend on internal structure
class BadUndoManager {
    saveState(editor: Editor): void {
        this.savedContent = editor.content; // Tight coupling
        this.savedPosition = editor.cursorPosition;
    }
}
```

### **The Solution: Caretaker/Originator/Memento Triad**

The Memento pattern creates three roles that preserve encapsulation while enabling state capture.

```typescript
/**
 * MEMENTO
 * Stores internal state of the Originator.
 * Protects against access by objects other than the Originator.
 */
interface EditorMemento {
    getContent(): string; // Read-only interface for Caretaker
    getMetadata(): { date: Date; label: string };
    // No setters! State is immutable once captured.
}

/**
 * CONCRETE MEMENTO (Hidden from clients)
 * Actual implementation with full state access.
 */
class ConcreteEditorMemento implements EditorMemento {
    constructor(
        private content: string,
        private cursorPosition: number,
        private selection: string,
        private date: Date = new Date()
    ) {}
    
    // Package-private or Originator-accessible methods
    getContent(): string { return this.content; }
    getCursorPosition(): number { return this.cursorPosition; }
    getSelection(): string { return this.selection; }
    getMetadata(): { date: Date; label: string } {
        return { date: this.date, label: `Saved at ${this.date.toLocaleTimeString()}` };
    }
}

/**
 * ORIGINATOR
 * Creates a memento containing a snapshot of its current internal state.
 * Uses the memento to restore its internal state.
 */
class Editor {
    private content: string = "";
    private cursorPosition: number = 0;
    private selection: string = "";
    
    // State modification methods
    type(text: string): void {
        this.content += text;
        this.cursorPosition += text.length;
        console.log(`Typed: "${text}"`);
    }
    
    select(start: number, end: number): void {
        this.selection = this.content.substring(start, end);
        this.cursorPosition = end;
        console.log(`Selected: "${this.selection}"`);
    }
    
    deleteSelection(): void {
        if (this.selection) {
            this.content = this.content.replace(this.selection, "");
            console.log(`Deleted selection`);
            this.selection = "";
        }
    }
    
    display(): void {
        console.log(`[Editor] Content: "${this.content}", Cursor: ${this.cursorPosition}, Selection: "${this.selection}"`);
    }
    
    // MEMENTO OPERATIONS
    /**
     * Creates memento containing current state.
     * Encapsulation preserved: Caretaker sees only Memento interface.
     */
    save(): EditorMemento {
        return new ConcreteEditorMemento(
            this.content,
            this.cursorPosition,
            this.selection
        );
    }
    
    /**
     * Restores state from memento.
     */
    restore(memento: EditorMemento): void {
        const concrete = memento as ConcreteEditorMemento; // Type-safe cast in real implementation
        this.content = concrete.getContent();
        this.cursorPosition = concrete.getCursorPosition();
        this.selection = concrete.getSelection();
        console.log("State restored");
    }
}

/**
 * CARETAKER
 * Responsible for the memento's safekeeping.
 * Never operates on or examines the contents of a memento.
 */
class HistoryManager {
    private history: EditorMemento[] = [];
    private currentIndex: number = -1;
    private maxHistory: number = 50; // Limit memory usage
    
    backup(editor: Editor): void {
        // Remove any redo states if we branch from middle of history
        if (this.currentIndex < this.history.length - 1) {
            this.history = this.history.slice(0, this.currentIndex + 1);
        }
        
        const memento = editor.save();
        this.history.push(memento);
        this.currentIndex++;
        
        // Enforce history limit
        if (this.history.length > this.maxHistory) {
            this.history.shift();
            this.currentIndex--;
        }
        
        console.log(`[History] Saved snapshot #${this.currentIndex}`);
    }
    
    undo(editor: Editor): boolean {
        if (this.currentIndex <= 0) {
            console.log("[History] Nothing to undo");
            return false;
        }
        
        this.currentIndex--;
        const memento = this.history[this.currentIndex];
        editor.restore(memento);
        console.log(`[History] Undone to snapshot #${this.currentIndex}`);
        return true;
    }
    
    redo(editor: Editor): boolean {
        if (this.currentIndex >= this.history.length - 1) {
            console.log("[History] Nothing to redo");
            return false;
        }
        
        this.currentIndex++;
        const memento = this.history[this.currentIndex];
        editor.restore(memento);
        console.log(`[History] Redone to snapshot #${this.currentIndex}`);
        return true;
    }
    
    getHistoryList(): string[] {
        return this.history.map((m, i) => 
            `${i === this.currentIndex ? '>' : ' '} ${i}: ${m.getMetadata().label}`
        );
    }
}

// Usage
const editor = new Editor();
const history = new HistoryManager();

// Initial state
editor.type("Hello ");
history.backup(editor);

editor.type("World");
editor.select(6, 11); // Select "World"
history.backup(editor);

editor.type("Universe"); // Replace selection
history.backup(editor);

editor.display(); // "Hello Universe"

// Undo operations
history.undo(editor); // Back to "Hello World" with selection
editor.display();

history.undo(editor); // Back to "Hello "
editor.display();

history.redo(editor); // Forward to "Hello World"
editor.display();

console.log("\nHistory:", history.getHistoryList());
```

**Code Explanation**:

- **Originator** (`Editor`): The object whose state needs saving. Creates mementos containing its state and can restore from them.
- **Memento** (`EditorMemento`): Stores Originator's state. Provides two interfaces: a **wide interface** (to Originator, allowing state restoration) and a **narrow interface** (to Caretaker, read-only metadata only).
- **Caretaker** (`HistoryManager`): Maintains the history of mementos. Never modifies mementos—only stores and retrieves them. Manages undo/redo logic.

**Encapsulation Preservation**:
The Caretaker (`HistoryManager`) only knows about the `EditorMemento` interface with `getContent()` and `getMetadata()`. It cannot access `cursorPosition` or `selection` directly, preserving `Editor`'s encapsulation. Only `Editor` (Originator) can access the full state via the ConcreteMemento.

### **Memory Optimization: Incremental Mementos**
For large objects, store only changes (deltas) rather than full state copies:

```typescript
class IncrementalMemento implements EditorMemento {
    private contentDelta: string; // Only what changed
    private positionDelta: number;
    private previousState: EditorMemento; // Chain for reconstruction
    
    constructor(changes: Partial<EditorState>, previous: EditorMemento) {
        this.contentDelta = changes.content || "";
        this.positionDelta = changes.cursorPosition || 0;
        this.previousState = previous;
    }
    
    restoreIncremental(): EditorState {
        const base = this.previousState.getContent();
        return {
            content: base + this.contentDelta,
            cursorPosition: this.positionDelta
        };
    }
}
```

---

## **10.3 Visitor Pattern: Adding Operations Without Modifying Classes**

### **Intent**
*Represent an operation to be performed on the elements of an object structure. Visitor lets you define a new operation without changing the classes of the elements on which it operates.*

### **The Problem**
Consider an AST (Abstract Syntax Tree) with `NumberExpression`, `AdditionExpression`, and `MultiplicationExpression` classes. Adding a new operation (pretty-printing, evaluation, type-checking) requires modifying every existing class, violating OCP:

```typescript
// ❌ VIOLATES OCP: Adding new operation requires modifying all classes
interface Expression {
    evaluate(): number;          // Operation 1
    print(): string;             // Operation 2 (added later - modified interface!)
    typeCheck(): Type;           // Operation 3 (modified again!)
    // Every new operation forces changes to all existing Expression classes
}

class NumberExpression implements Expression {
    constructor(private value: number) {}
    
    evaluate(): number { return this.value; }
    print(): string { return this.value.toString(); }
    typeCheck(): Type { return Type.NUMBER; }
    // Must implement every operation even if irrelevant
}
```

### **The Solution: Double Dispatch**

The Visitor pattern separates operations from object structure by moving operations into separate classes (Visitors) that "visit" elements.

```typescript
/**
 * ELEMENT INTERFACE
 * Declares an accept method that takes a visitor as argument.
 */
interface Expression {
    accept(visitor: ExpressionVisitor): void;
}

/**
 * VISITOR INTERFACE
 * Declares visit operations for each concrete element class.
 * The operation name (visitNumber, visitAdd) identifies the argument type.
 */
interface ExpressionVisitor {
    visitNumber(expr: NumberExpression): void;
    visitAddition(expr: AdditionExpression): void;
    visitMultiplication(expr: MultiplicationExpression): void;
}

/**
 * CONCRETE ELEMENTS
 * Implement accept method: visitor.visitSpecificType(this)
 */
class NumberExpression implements Expression {
    constructor(public value: number) {} // public for visitor access
    
    accept(visitor: ExpressionVisitor): void {
        visitor.visitNumber(this);
    }
}

class AdditionExpression implements Expression {
    constructor(
        public left: Expression,
        public right: Expression
    ) {}
    
    accept(visitor: ExpressionVisitor): void {
        visitor.visitAddition(this);
    }
}

class MultiplicationExpression implements Expression {
    constructor(
        public left: Expression,
        public right: Expression
    ) {}
    
    accept(visitor: ExpressionVisitor): void {
        visitor.visitMultiplication(this);
    }
}

/**
 * CONCRETE VISITORS
 * Implement each operation declared by Visitor interface.
 * Each visitor provides a different operation over the structure.
 */
class EvaluateVisitor implements ExpressionVisitor {
    private result: number = 0;
    
    visitNumber(expr: NumberExpression): void {
        this.result = expr.value;
    }
    
    visitAddition(expr: AdditionExpression): void {
        // Post-order traversal: visit children first
        expr.left.accept(this);
        const leftVal = this.result;
        expr.right.accept(this);
        const rightVal = this.result;
        this.result = leftVal + rightVal;
    }
    
    visitMultiplication(expr: MultiplicationExpression): void {
        expr.left.accept(this);
        const leftVal = this.result;
        expr.right.accept(this);
        const rightVal = this.result;
        this.result = leftVal * rightVal;
    }
    
    getResult(): number {
        return this.result;
    }
}

class PrintVisitor implements ExpressionVisitor {
    private result: string = "";
    
    visitNumber(expr: NumberExpression): void {
        this.result = expr.value.toString();
    }
    
    visitAddition(expr: AdditionExpression): void {
        expr.left.accept(this);
        const left = this.result;
        expr.right.accept(this);
        const right = this.result;
        this.result = `(${left} + ${right})`;
    }
    
    visitMultiplication(expr: MultiplicationExpression): void {
        expr.left.accept(this);
        const left = this.result;
        expr.right.accept(this);
        const right = this.result;
        this.result = `(${left} * ${right})`;
    }
    
    getResult(): string {
        return this.result;
    }
}

// Usage: Expression structure remains stable
const expression = new AdditionExpression(
    new NumberExpression(5),
    new MultiplicationExpression(
        new NumberExpression(3),
        new NumberExpression(2)
    )
);

// Operation 1: Evaluate
const evaluator = new EvaluateVisitor();
expression.accept(evaluator);
console.log(`Result: ${evaluator.getResult()}`); // 11

// Operation 2: Print (no changes to Expression classes needed!)
const printer = new PrintVisitor();
expression.accept(printer);
console.log(`Expression: ${printer.getResult()}`); // (5 + (3 * 2))

// Operation 3: Type checking (new visitor, no modifications)
class TypeCheckVisitor implements ExpressionVisitor {
    private type: string = "";
    
    visitNumber(expr: NumberExpression): void {
        this.type = "number";
    }
    
    visitAddition(expr: AdditionExpression): void {
        expr.left.accept(this);
        expr.right.accept(this);
        this.type = "number"; // Simplified
    }
    
    visitMultiplication(expr: MultiplicationExpression): void {
        // ... similar
        this.type = "number";
    }
}
```

**Code Explanation**:

- **Visitor** (`ExpressionVisitor`): Interface declaring visit methods for each concrete element type. Each method represents an operation on that specific element type.
- **ConcreteVisitor** (`EvaluateVisitor`, `PrintVisitor`): Implements the operations declared by Visitor. Each visitor provides a distinct algorithm over the object structure.
- **Element** (`Expression`): Interface declaring `accept(visitor)`, allowing visitors to operate on it.
- **ConcreteElement** (`NumberExpression`, etc.): Implements `accept` by calling `visitor.visitSpecificType(this)`. This is **double dispatch**: first dispatch selects the element's `accept`, second dispatch selects the visitor's specific `visit` method.

**Double Dispatch Mechanism**:
1.  Client calls `element.accept(visitor)`
2.  Element calls `visitor.visitConcreteElement(this)` (first dispatch resolved by element type)
3.  Visitor executes `visitConcreteElement` (second dispatch resolved by visitor type)

**Trade-offs**:
- **Pro**: Adding new operations requires only new Visitor classes (OCP satisfied for operations).
- **Con**: Adding new Element classes requires modifying all existing Visitors to add `visitNewElement` method.

### **When to Use Visitor**
- Object structure contains many classes with differing interfaces.
- Many distinct operations needed on these classes.
- Classes rarely change, but operations change frequently.
- You need to accumulate state across the traversal (e.g., counting nodes, collecting errors).

---

## **10.4 Interpreter Pattern: Evaluating Language Sentences**

### **Intent**
*Given a language, define a representation for its grammar along with an interpreter that uses the representation to interpret sentences in the language.*

### **The Problem**
Domain-specific languages (DSL) for configuration, search queries, or mathematical expressions require parsing and evaluation. Hard-coding parsers creates unmaintainable monolithic classes.

### **The Solution: Grammar as Class Hierarchy**

The Interpreter pattern represents grammar rules as classes, building an Abstract Syntax Tree (AST) that can be evaluated.

```typescript
/**
 * ABSTRACT EXPRESSION
 * Declares an interpret operation common to all nodes in AST.
 */
interface Expression {
    interpret(context: Context): number;
    toString(): string;
}

/**
 * CONTEXT
 * Contains global information needed during interpretation.
 */
class Context {
    private variables: Map<string, number> = new Map();
    
    setVariable(name: string, value: number): void {
        this.variables.set(name, value);
    }
    
    getVariable(name: string): number {
        const val = this.variables.get(name);
        if (val === undefined) throw new Error(`Undefined variable: ${name}`);
        return val;
    }
}

/**
 * TERMINAL EXPRESSION
 * Implements interpret for terminal symbols (leaf nodes).
 */
class NumberExpression implements Expression {
    constructor(private value: number) {}
    
    interpret(context: Context): number {
        return this.value;
    }
    
    toString(): string {
        return this.value.toString();
    }
}

class VariableExpression implements Expression {
    constructor(private name: string) {}
    
    interpret(context: Context): number {
        return context.getVariable(this.name);
    }
    
    toString(): string {
        return this.name;
    }
}

/**
 * NON-TERMINAL EXPRESSIONS
 * Implements interpret for grammar rules (composite nodes).
 */
class AdditionExpression implements Expression {
    constructor(
        private left: Expression,
        private right: Expression
    ) {}
    
    interpret(context: Context): number {
        return this.left.interpret(context) + this.right.interpret(context);
    }
    
    toString(): string {
        return `(${this.left.toString()} + ${this.right.toString()})`;
    }
}

class SubtractionExpression implements Expression {
    constructor(
        private left: Expression,
        private right: Expression
    ) {}
    
    interpret(context: Context): number {
        return this.left.interpret(context) - this.right.interpret(context);
    }
    
    toString(): string {
        return `(${this.left.toString()} - ${this.right.toString()})`;
    }
}

class MultiplicationExpression implements Expression {
    constructor(
        private left: Expression,
        private right: Expression
    ) {}
    
    interpret(context: Context): number {
        return this.left.interpret(context) * this.right.interpret(context);
    }
    
    toString(): string {
        return `${this.left.toString()} * ${this.right.toString()}`;
    }
}

// Advanced: Conditional expression
class ConditionalExpression implements Expression {
    constructor(
        private condition: Expression,
        private trueExpr: Expression,
        private falseExpr: Expression
    ) {}
    
    interpret(context: Context): number {
        const condVal = this.condition.interpret(context);
        return condVal !== 0 
            ? this.trueExpr.interpret(context) 
            : this.falseExpr.interpret(context);
    }
    
    toString(): string {
        return `if ${this.condition.toString()} then ${this.trueExpr.toString()} else ${this.falseExpr.toString()}`;
    }
}

// Parser (simplified - real parsers use recursive descent or parser generators)
class ExpressionParser {
    static parse(expression: string): Expression {
        // Simplified: Assumes space-separated tokens in postfix notation
        // Real implementation would use tokenization and recursive descent
        const tokens = expression.split(' ');
        const stack: Expression[] = [];
        
        for (const token of tokens) {
            if (!isNaN(Number(token))) {
                stack.push(new NumberExpression(Number(token)));
            } else if (['+', '-', '*'].includes(token)) {
                const right = stack.pop()!;
                const left = stack.pop()!;
                
                switch(token) {
                    case '+': stack.push(new AdditionExpression(left, right)); break;
                    case '-': stack.push(new SubtractionExpression(left, right)); break;
                    case '*': stack.push(new MultiplicationExpression(left, right)); break;
                }
            } else {
                stack.push(new VariableExpression(token));
            }
        }
        
        return stack[0];
    }
}

// Usage
const context = new Context();
context.setVariable("x", 10);
context.setVariable("y", 5);

// Build AST manually or via parser
const expression = new AdditionExpression(
    new MultiplicationExpression(
        new VariableExpression("x"),
        new NumberExpression(2)
    ),
    new VariableExpression("y")
);

console.log(`Expression: ${expression.toString()}`); // ((x * 2) + y)
console.log(`Result: ${expression.interpret(context)}`); // 25

// Parse from string (postfix: "x 2 * y +")
const parsed = ExpressionParser.parse("x 2 * y +");
console.log(`Parsed: ${parsed.interpret(context)}`); // 25

// Complex: (x > 0) ? x : 0
const conditional = new ConditionalExpression(
    new VariableExpression("x"), // simplified: truthy check
    new VariableExpression("x"),
    new NumberExpression(0)
);
console.log(`Conditional: ${conditional.interpret(context)}`); // 10
```

**Code Explanation**:

- **AbstractExpression** (`Expression`): Declares `interpret(context)`. Common interface for all nodes in AST.
- **TerminalExpression** (`NumberExpression`, `VariableExpression`): Leaf nodes representing literals or variables. Directly return values.
- **NonTerminalExpression** (`AdditionExpression`, etc.): Composite nodes containing sub-expressions. Implement `interpret` by recursively interpreting children and combining results.
- **Context** (`Context`): Stores global state (variable values) accessible during interpretation.
- **Client**: Builds AST (via parser or manually) and initiates interpretation by calling `interpret` on root node.

**Applicability**:
- Grammar is relatively simple (complex grammars need parser generators like ANTLR).
- Efficiency is not critical (tree traversal overhead).
- When you need to interpret simple DSLs (configuration languages, query languages).

**Connection to Composite Pattern**:
Interpreter uses Composite structure (expressions contain sub-expressions). The AST is a Composite tree; interpretation is a traversal operation.

---

## **Chapter Summary**
This chapter completed the Behavioral patterns catalog:

1.  **State**: Encapsulates state-specific behavior in separate classes. Objects appear to change class as they transition states. Eliminates complex conditional state logic. **Status: Essential for state machines, workflows, and lifecycle management.**

2.  **Memento**: Captures and restores object state without violating encapsulation. Enables undo/redo functionality through Caretaker-Originator collaboration. **Status: Critical for editor applications and transaction rollback systems.**

3.  **Visitor**: Separates operations from object structure, allowing new operations without modifying element classes. Uses double dispatch for type-safe traversal. **Status: Specialized for compilers, AST processing, and complex object structures with stable class hierarchies.**

4.  **Interpreter**: Represents grammar as class hierarchy, building ASTs for sentence evaluation. Suitable for simple DSLs. **Status: Specialized; complex languages require dedicated parser generators.**

**Key Distinctions**:
- **State** manages behavior variation through internal state changes.
- **Memento** externalizes state for restoration while preserving encapsulation.
- **Visitor** adds operations to stable class hierarchies.
- **Interpreter** evaluates sentences according to defined grammar.

---

## **Next Chapter Preview**
**Chapter 11: Patterns in modern languages** - Exploring how these classic patterns are implemented and evolved in languages like TypeScript, Python, and Rust, leveraging language features for more concise and powerful designs.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='9. communication_and_coordination.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../3. modern_adaptations_functional_patterns/11. patterns_in_modern_language_features.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
